# **Online payments Fraud Detection with Machine Learning**

`The introduction of online payment systems has helped a lot in the ease of payments.But , at the same time,it increased in payment frauds.Online payment frauds can happen with anyone using any payment system,especially while making payments using a credit card.That is why detecting online payment fraud is very important for credit card companies to ensure that the customers are not getting charged for the products and services they never paid`

`To identify online payment fraud with machine learning, we need to train a machine learning model for classifying fraudulent and non-fraudulent payments. For this, we need a dataset containing information about online payment fraud, so that we can understand what type of transactions lead to fraud. For this task, I collected a dataset from Kaggle, which contains historical information about fraudulent transactions which can be used to detect fraud in online payments. Below are all the columns from the dataset I’m using here:`

step: represents a unit of time where 1 step equals 1 hour

type: type of online transaction

amount: the amount of the transaction

nameOrig: customer starting the transaction

oldbalanceOrg: balance before the transaction

newbalanceOrig: balance after the transaction

nameDest: recipient of the transaction

oldbalanceDest: initial balance of recipient before the transaction

newbalanceDest: the new balance of recipient after the transaction

isFraud: fraud transaction

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/Machine Learning/PS_20174392719_1491204439457_log.csv")
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


- ` let’s have a look at whether this dataset has any null values or not:`

In [2]:
data.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


`So this dataset does not have any null values. Before moving forward, now, let’s have a look at the type of transaction mentioned in the dataset:`

In [3]:
# exploring transaction type
print(data.type.value_counts())

type
CASH_OUT    2237500
PAYMENT     2151495
CASH_IN     1399284
TRANSFER     532909
DEBIT         41432
Name: count, dtype: int64


In [4]:
type = data['type'].value_counts()
transactions = type.index
quantity = type.values

import plotly.express as px
figure = px.pie(data,
                values = quantity,
                names = transactions , hole = 0.5,
                title = 'Distribution of transaction type')
figure.show()

`let’s have a look at the correlation between the features of the data with the isFraud column:`

In [7]:
# checking corelation
corelation = data.select_dtypes(exclude='object').corr()
print(corelation['isFraud'].sort_values(ascending = False))

isFraud           1.000000
amount            0.076688
isFlaggedFraud    0.044109
step              0.031578
oldbalanceOrg     0.010154
newbalanceDest    0.000535
oldbalanceDest   -0.005885
newbalanceOrig   -0.008148
Name: isFraud, dtype: float64


`Now let’s transform the categorical features into numerical. Here I will also transform the values of the isFraud column into No Fraud and Fraud labels to have a better understanding of the output:`

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['type'] = le.fit_transform(data['type'])
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,3,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,4,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,1,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,3,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [9]:
data['isFraud'] = le.fit_transform(data['isFraud'])

In [10]:
# data["isFraud"] = data["isFraud"].map({0: "No Fraud", 1: "Fraud"})
# print(data.head())

`Now let’s train a classification model to classify fraud and non-fraud transactions. Before training the model, I will split the data into training and test sets:`

In [11]:
from sklearn.model_selection import train_test_split
x = np.array(data[['type' , 'amount' , 'oldbalanceOrg' , 'newbalanceOrig']])
y = np.array(data[['isFraud']])

In [12]:
from sklearn.tree import DecisionTreeClassifier
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.10, random_state=42)
model = DecisionTreeClassifier()
model.fit(xtrain, ytrain)

DecisionTreeClassifier()

In [13]:
print(model.score(xtest, ytest))

0.9997280994307377


In [14]:
#predictions
#features = ['type' , 'amount' , 'oldbalanceOrg' , 'newbalanceOrig']
features = np.array([[4 , 9000.60 , 9000.60 , 0.0]])
print(model.predict(features))

[1]


In [15]:
import pickle

with open('model.pkl' , 'wb') as file:
  pickle.dump(model , file)